# <center>Generating Music Using LSTM Cells</center>

This workbook will implement modified code from [this](https://github.com/corynguyen19/midi-lstm-gan) GitHub repo.

The idea is to read in MIDI files and convert them to arrays of notes. Then an RNN will be trained to predict the next note. Finally, music is generated by feeding a random string of notes to the RNN and having it iteratively predict the next note to form a song one note at a time.

### Things to test tomorrow:

* Add batch normalization - Loss won't go below ~3
* Add an extra layer (RNN and dense) - Meh
* Normalize between -1 to 1 - Didn't do much
* Decrease step size - MUCH better results with step size of 1
* Mess with batch size - Smaller seems better
* Mash-up 2 songs? - Works surprisingly well
* Converges? - No, different inputs should produce different tracks using the same model
* Try transfer learning and training the last few layers on this data?


### Visualizations
Cross correlations

In [28]:
# Imports
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
from music21 import converter, instrument, note, chord, stream, duration
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import Callback, ModelCheckpoint
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import CuDNNLSTM, LSTM, Bidirectional
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, History

## Loading and Cleaning the Data

First, I will load all the notes from the midi files

In [29]:
def get_notes(path):
    """
        Gets all notes and chords from midi files
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
        print("Parsing %s" % file)
        
        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append([str(element.pitch), element.offset, element.duration])
            elif isinstance(element, chord.Chord):
                song_note = '.'.join(str(n) for n in element.normalOrder)
                song.append([song_note, element.offset, element.duration])
        notes.append(song)

    return notes

def get_notes_with_key(path, filter_key, mode):
    """
        Gets all notes and chords from midi files
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
#         Only use music of the same key
        key = midi.analyze('key')
        if(mode==0):
            key_string = str(key.tonic.name)
        elif(mode==1):
            key_string = str(key.mode)
        else:
            key_string = str(key.tonic.name + key.mode)
            
        if(key_string==filter_key):
            print("Parsing %s" % file)

            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse() 
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    song.append([str(element.pitch), element.offset, element.duration])
                elif isinstance(element, chord.Chord):
                    song_note = '.'.join(str(n) for n in element.normalOrder)
                    song.append([song_note, element.offset, element.duration])
            notes.append(song)

    return notes

# def get_notes_with_key(path, filter_key, mode):
#     """
#         Gets all notes and chords from midi files where the key matches the string input

#         Parameters
#         ----------
#         path : str
#             The path to the file
#         filter_key : str
#             The string to filter the key on
#         mode : int
#             The type of key used where:
#                 0 - key
#                 1 - major/minor
#                 else - key and major/minor
#     """
#     notes = []

#     for file in glob.glob(path + "*.mid"):        
#         song = []
#         midi = converter.parse(file)
        
#         # Only use music of the same key
#         key = midi.analyze('key')
#         if(mode==0):
#             key_string = str(key.tonic.name)
#         elif(mode==1):
#             key_string = str(key.mode)
#         else:
#             key_string = str(key.tonic.name + key.mode)
            
#         if(key_string==filter_key):
#             print("Parsing %s" % file)
#             notes_to_parse = None

#             try: # file has instrument parts
#                 s2 = instrument.partitionByInstrument(midi)
#                 notes_to_parse = s2.parts[0].recurse() 
#             except: # file has notes in a flat structure
#                 notes_to_parse = midi.flat.notes

#             for element in notes_to_parse:
#                 if isinstance(element, note.Note):
#                     song.append(str(element.pitch))
#                 elif isinstance(element, chord.Chord):
#                     song.append('.'.join(str(n) for n in element.normalOrder))

#     return notes

""" Train a Neural Network to generate music """
# Get notes from midi files
input_dir_choice = 6
input_dir_names = ["test", "Pokemon", "LoZ OOT", "Pokemon GSC", "Pokemon Battle", "Undertale", "ABBA"]

input_path = "../" + input_dir_names[input_dir_choice] + " MIDIs/"
# example of each mode: 0 - C, 1 - major, 2 - Cmajor
# notes = get_notes_with_key(input_path, "minor", 1)
notes = get_notes(input_path)

Parsing ../ABBA MIDIs\ABBA.Alaska.mid
Parsing ../ABBA MIDIs\ABBA.Angel Eyes K.mid
Parsing ../ABBA MIDIs\ABBA.Another Town, Another Train K.mid
Parsing ../ABBA MIDIs\ABBA.Arrival.mid
Parsing ../ABBA MIDIs\ABBA.As good as new.mid
Parsing ../ABBA MIDIs\ABBA.Bang a Boomerang K.mid
Parsing ../ABBA MIDIs\ABBA.Chiquitita K.mid
Parsing ../ABBA MIDIs\ABBA.Dance (While The Music Still Goes On) K.mid
Parsing ../ABBA MIDIs\ABBA.Dancing queen K.mid
Parsing ../ABBA MIDIs\ABBA.Does your mother know K.mid
Parsing ../ABBA MIDIs\ABBA.Dum Dum Diddle K.mid
Parsing ../ABBA MIDIs\ABBA.Eagle.mid
Parsing ../ABBA MIDIs\ABBA.Fernando K.mid
Parsing ../ABBA MIDIs\ABBA.Gimme gimme gimme L.mid
Parsing ../ABBA MIDIs\ABBA.Hamlet III.mid
Parsing ../ABBA MIDIs\ABBA.Happy new year.mid
Parsing ../ABBA MIDIs\ABBA.Hasta Manana K.mid
Parsing ../ABBA MIDIs\ABBA.He Is Your Brother K.mid
Parsing ../ABBA MIDIs\ABBA.Hey Hey Helen K.mid
Parsing ../ABBA MIDIs\ABBA.Hole in your soul.mid
Parsing ../ABBA MIDIs\ABBA.I am the city.mid


I will now use an algo to determine the key of each song

In [30]:
def print_key(path):
    key_count = dict()
    for file in glob.glob(path + "*.mid"):
        print("Parsing %s" % file)
        
        song = []
        midi = converter.parse(file)
        
        key = midi.analyze('key')
        key_string = key.tonic.name + key.mode
        if (key_string in key_count): 
            key_count[key_string] += 1
        else: 
            key_count[key_string] = 1
        print(key.tonic.name, key.mode)
    return key_count

key_count = print_key(input_path)
# key_count = print_key("../Undertale MIDIs/")
key_count

Parsing ../ABBA MIDIs\ABBA.Alaska.mid
A minor
Parsing ../ABBA MIDIs\ABBA.Angel Eyes K.mid
B major
Parsing ../ABBA MIDIs\ABBA.Another Town, Another Train K.mid
C major
Parsing ../ABBA MIDIs\ABBA.Arrival.mid
D major
Parsing ../ABBA MIDIs\ABBA.As good as new.mid
E major
Parsing ../ABBA MIDIs\ABBA.Bang a Boomerang K.mid
F major
Parsing ../ABBA MIDIs\ABBA.Chiquitita K.mid
A major
Parsing ../ABBA MIDIs\ABBA.Dance (While The Music Still Goes On) K.mid
C major
Parsing ../ABBA MIDIs\ABBA.Dancing queen K.mid
A minor
Parsing ../ABBA MIDIs\ABBA.Does your mother know K.mid
C major
Parsing ../ABBA MIDIs\ABBA.Dum Dum Diddle K.mid
E major
Parsing ../ABBA MIDIs\ABBA.Eagle.mid
A minor
Parsing ../ABBA MIDIs\ABBA.Fernando K.mid
A major
Parsing ../ABBA MIDIs\ABBA.Gimme gimme gimme L.mid
D minor
Parsing ../ABBA MIDIs\ABBA.Hamlet III.mid
F major
Parsing ../ABBA MIDIs\ABBA.Happy new year.mid
A major
Parsing ../ABBA MIDIs\ABBA.Hasta Manana K.mid
G minor
Parsing ../ABBA MIDIs\ABBA.He Is Your Brother K.mid
A maj

{'Aminor': 4,
 'Bmajor': 3,
 'Cmajor': 7,
 'Dmajor': 9,
 'Emajor': 6,
 'Fmajor': 2,
 'Amajor': 6,
 'Dminor': 3,
 'Gminor': 3,
 'Gmajor': 1,
 'B-major': 3,
 'Eminor': 2,
 'Bminor': 1,
 'Cminor': 1,
 'B-minor': 1}

Next, I will find all possible notes and use this to determine how to alter the data to a machine readable format.

In [31]:
possibleNotes = set([item[0] for sublist in notes for item in sublist])

# Processing for offsets
possibleOffsets = []
possibleDurations = []

# For each song
for index, song in enumerate(notes):
    song_length = len(song)
    
    # For each note, calculate the difference in offset between this and the previous note
    song_offsets = []
    song_durations = []
    for idx in range(song_length):
        offset = offset = round(song[idx][1] - song[idx - 1][1], 3) if idx > 1 else 0.0
        song_offsets.append(offset)
        if offset not in possibleOffsets:
            possibleOffsets.append(offset)
        
        duration = song[idx][2].quarterLength
        song_durations.append(duration)
        if duration not in possibleDurations:
            possibleDurations.append(duration)
            
    # Update the notes to reflect this
    for idx in range(song_length):
        notes[index][idx][1] = song_offsets[idx]
        notes[index][idx][2] = song_durations[idx]

n_notes = len(possibleNotes)
n_offset = len(possibleOffsets)
n_duration = len(possibleDurations)


possibleNotes = np.array(list(possibleNotes))
possibleOffsets = np.array(list(possibleOffsets))
possibleDurations = np.array(list(possibleDurations))
notes = np.array([list([list(subsublist) for subsublist in sublist]) for sublist in notes])
len(possibleNotes), len(possibleOffsets), len(possibleDurations)

(393, 60, 87)

Now I will prepare the sequences of notes by looking at each song individually. I will first grab an arrays of size **sequence_length** with a stride of **step_size** from each song. Then I will map the chords to integers so the model can learn from that and normalize the input between 0-1.

In [32]:
def prepare_sequences(notes, possibleNotes, possibleOffsets, possibleDurations):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    step_size = 1

    # create a dictionary to map pitches to integers
    pitchnames = sorted(possibleNotes)
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    # create a dictionary to map offset to integers
    offsetnames = sorted(possibleOffsets)
    offset_to_int = dict((offset, number) for number, offset in enumerate(offsetnames))
    
    # create a dictionary to map duration to integers
    durationnames = sorted(possibleDurations)
    duration_to_int = dict((duration, number) for number, duration in enumerate(durationnames))
    
    # find number of each possible choice for normalization
    n_notes = len(possibleNotes)
    n_offset = len(possibleOffsets)
    n_duration = len(possibleDurations)

    network_input = []
    network_output_notes = []
    network_output_offset = []
    network_output_duration = []


    # create input sequences and the corresponding outputs
    for song in notes:
        for i in range(0, len(song) - sequence_length, step_size):
            sequence_in = song[i:i + sequence_length]
            sequence_out = song[i + sequence_length]
            network_input.append([np.array([note_to_int[row[0]] / float(n_notes), offset_to_int[row[1]] / float(n_offset), duration_to_int[row[2]] / float(n_duration)]) for row in sequence_in])
            network_output_notes.append(np.array([note_to_int[sequence_out[0]]]))
            network_output_offset.append(np.array([offset_to_int[sequence_out[1]]]))
            network_output_duration.append(np.array([duration_to_int[sequence_out[2]]]))


    # reshape the input into a format compatible with LSTM layers
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 3))

    # Make one-hot-encoding
    network_output_notes = np_utils.to_categorical(network_output_notes, num_classes=n_notes)
    network_output_offset = np_utils.to_categorical(network_output_offset, num_classes=n_offset)
    network_output_duration = np_utils.to_categorical(network_output_duration, num_classes=n_duration)


    return (network_input, network_output_notes, network_output_offset, network_output_duration)

network_input, network_output_notes, network_output_offset, network_output_duration = prepare_sequences(notes, possibleNotes, possibleOffsets, possibleDurations)
network_input.shape

(36005, 100, 3)

## Constructing the model

I will now construct the model using CuDNNLSTM cells because they are significantly faster than regular LSTM cells due to being optimized for CuDA. I will have two CuDNNLSTM layers, followed by two dense layers and a final softmax activation layer to output the most probable result.

Hyperparameters:
* Optimizer - ADAM because it is considered one of the best
* Loss - categorical_crossentropy because it penalizes wrong predictions of multi-class problems best
* Epochs - More epochs are generally better as long as they don't overfit. I track loss over time and have checkpoints every 5 epochs so this will not be a problem
* Batch Size - This determines how many instances should be considered in each batch. Realistically, each different song would interfere with the other so I will reduce this.
    * Smaller Batch Size seems to add more variation

In [33]:
from keras import Input
from keras.models import Model

def create_network(network_input, n_notes, n_offset, n_duration):
    """ create the structure of the neural network """
    input = Input(shape=(network_input.shape[1], network_input.shape[2]))
    lstm_1 = CuDNNLSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True)(input)
    dropout_1 = Dropout(0.2)(lstm_1)
    lstm_2 = Bidirectional(CuDNNLSTM(512, return_sequences=True))(dropout_1)
    dropout_2 = Dropout(0.2)(lstm_2)
    lstm_3 = Bidirectional(CuDNNLSTM(512))(dropout_2)
    dense_1 = Dense(256)(lstm_3)
    dropout_3 = Dropout(0.2)(dense_1)
    dense_2 = Dense(3, input_shape=(n_notes, n_offset, n_duration))(dropout_3)
    output_notes = Dense(n_notes, activation='softmax')(dropout_3)
    output_offset = Dense(n_offset, activation='softmax')(dropout_3)
    output_duration = Dense(n_duration, activation='softmax')(dropout_3)
    
    model = Model(inputs=input, outputs=[output_notes, output_offset, output_duration])
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], optimizer='adam', loss_weights=[1., 1., 1.])
    return model

# Set up the model
model = create_network(network_input, n_notes, n_offset, n_duration)
history = History()

# Save on each epoch (because training isn't cheap!!!) and can use this to generate music for each checkpoint
outputDest = '../output/LSTM_' + input_dir_names[input_dir_choice] + '_' + str(int(time.time())) + '/'
if not os.path.exists(outputDest):
    os.makedirs(outputDest)

cp_callback = ModelCheckpoint(filepath=outputDest + "LSTMmodel_weights_{epoch:02d}.hdf5",
                              save_weights_only=True,
                              verbose=1,
                              period=3)

# Set parameters
n_epochs = 50
batch_size = 80
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100, 3)       0                                            
__________________________________________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)        (None, 100, 512)     1058816     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 100, 512)     0           cu_dnnlstm_4[0][0]               
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 100, 1024)    4202496     dropout_4[0][0]                  
____________________________________________________________________________________________

## Training the Model

I will save the final model, but keep checkpoints along the way to avoid overfitting and also use these to generate different midis.

In [34]:
model.fit(network_input, [network_output_notes, network_output_offset, network_output_duration], callbacks=[history, cp_callback], epochs=n_epochs, batch_size=batch_size)
model.save(outputDest + 'LSTMmodel_final.h5')

Epoch 1/50
36005/36005 [==============================] - 116s 3ms/step - loss: 8.1980 - dense_8_loss: 4.6529 - dense_9_loss: 1.3484 - dense_10_loss: 2.1967
Epoch 2/50
36005/36005 [==============================] - 99s 3ms/step - loss: 7.4275 - dense_8_loss: 4.2130 - dense_9_loss: 1.1731 - dense_10_loss: 2.0415
Epoch 3/50
36005/36005 [==============================] - 100s 3ms/step - loss: 6.5835 - dense_8_loss: 3.7179 - dense_9_loss: 0.9772 - dense_10_loss: 1.8884

Epoch 00003: saving model to ../output/LSTM_ABBA_1570771432/LSTMmodel_weights_03.hdf5
Epoch 4/50
36005/36005 [==============================] - 99s 3ms/step - loss: 5.5788 - dense_8_loss: 3.1222 - dense_9_loss: 0.7659 - dense_10_loss: 1.6907
Epoch 5/50
36005/36005 [==============================] - 99s 3ms/step - loss: 4.7277 - dense_8_loss: 2.6351 - dense_9_loss: 0.6027 - dense_10_loss: 1.4900
Epoch 6/50
36005/36005 [==============================] - 99s 3ms/step - loss: 4.0636 - dense_8_loss: 2.2394 - dense_9_loss: 0.4970

KeyboardInterrupt: 

In [38]:
# Plot the model losses
pd.DataFrame(history.history).plot()
plt.savefig(outputDest + 'LSTM_Loss_per_Epoch.png', transparent=True)
plt.close()

## Generating Music

I will now use the model to generate music by feeding it a random string of notes and have it predict the next one, then have it predict the one after that until a full song has been generated.

In [39]:
model.load_weights(outputDest + "LSTMmodel_weights_21.hdf5")

In [40]:
def generate_notes(model, network_input, possibleNotes, possibleOffsets, possibleDurations):
    """ Generate notes from the neural network based on a sequence of notes """
    # create a dictionary to map pitches to integers
    pitchnames = sorted(possibleNotes)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # create a dictionary to map offset to integers
    offsetnames = sorted(possibleOffsets)
    int_to_offset = dict((number, offset) for number, offset in enumerate(offsetnames))
    
    # create a dictionary to map duration to integers
    durationnames = sorted(possibleDurations)
    int_to_duration = dict((number, duration) for number, duration in enumerate(durationnames))
    
    # find number of each possible choice for normalization
    n_notes = len(possibleNotes)
    n_offset = len(possibleOffsets)
    n_duration = len(possibleDurations)
    
    # choose a random point to start
    start = np.random.randint(0, len(network_input)-1)
    pattern = network_input[start]
#     np.random.shuffle(pattern)
    sequence_length = pattern.shape[0]
    n_dim = pattern.shape[1]
    
    prediction_output = []
    
    # generate 500 notes
    for note_index in range(600):
        prediction_input = np.reshape(pattern, (1, sequence_length, n_dim))
        prediction_input = prediction_input

        prediction = model.predict(prediction_input, verbose=0)
        
        note_int = np.argmax(prediction[0])
        note_normalized = note_int / float(n_notes)
        note = int_to_note[note_int]
        
        offset_int = np.argmax(prediction[1])
        offset_normalized = offset_int / float(n_offset)
        offset = int_to_offset[offset_int]
                
        duration_int = np.argmax(prediction[2])
        duration_normalized = duration_int / float(n_duration)
        duration = int_to_duration[duration_int]
        
        result = np.array([note_normalized, offset_normalized, duration_normalized])
        full_prediction = np.array([note, offset, duration])
        
        prediction_output.append(full_prediction)
        pattern = np.append(pattern, result)
        pattern = pattern[3:len(pattern)]
        
    print([str(x[0]) for x in prediction_output])
    
    return prediction_output[100:]

# prediction_output = generate_notes(model, network_input, possibleNotes, possibleOffsets, possibleDurations)

Next, I will create a midi using these notes and save to a file

In [41]:
from music21 import duration as D

def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    count = 0
    for pattern in prediction_output:
        note_str = pattern[0]
        offset_str = pattern[1]
        duration_str = pattern[2]
        if "#-" in note_str:# To fix a rare exception using 2 accidentals
            continue
        # pattern is a chord
        if ('.' in note_str) or note_str.isdigit():
            notes_in_chord = note_str.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            new_note.duration = D.Duration(float(duration_str))
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(note_str)
            new_note.offset = offset
            new_note.duration = D.Duration(float(duration_str))
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += (float(prediction_output[count + 1][1])) if (count + 1 < len(prediction_output)) else 0
        count += 1

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))
    
# 
# create_midi(prediction_output, outputDest + 'LSTM_output_finalX')
# create_midi(prediction_output, 'lavender')

Or a batch production could be performed of the final model

In [42]:
album_length = 10
for count_output in range(album_length):
    prediction_output = generate_notes(model, network_input, possibleNotes, possibleOffsets, possibleDurations)
    create_midi(prediction_output, outputDest + 'LSTM_output_final' + str(count_output))
    print(f"Created at {outputDest + 'LSTM_output_final' + str(count_output)}")

['C#2', 'A4', '1.4', '1.4', '1.4', '1.4.6', 'E4', '1.4', '1.4', '1.4', '1.4', '0.5.6', '2.5', 'C#2', 'A4', 'E4', '1.4', '1.4', '1.4', 'E4', '1.4', '1.4', '1.4', '0.2.5', '0.5.6', '2.5', 'D2', 'A4', '2.5', '2.5', '2.5.6', '10.1.4', 'A4', '2.5', '2.5', '2.5', '2.5', '2.5.6', '0.5.6', '2.5', 'D2', 'D4', '2.5', '2.5', '2.5', '5.6', 'A4', '2.5', '2.5', '2.5', '2.5.6', '0.5.6', 'D2', 'D4', '2.5', '2.5', '2.5.6', '2.5.9', '2.5', '2.5', '5.6', '2.5', '2.5', 'D2', '2.5', '2.5', 'D2', 'D4', '2.5', 'D2', '4.7', '2.5', '2.5', '2.5.6', '5.6', '2.5', '1.4', '2.5', '2.5', '0.5.6', 'D4', 'A4', 'D4', '2.5', '2.5', '2.5.6', '5.6', '4.7', '1.4', '2.5', '2.5', '0.5.6', 'D2', '9.10', 'A4', '2.5', '4.7', '4.6.7', '5.6', '2.5', '4.6.7', '4.7', '4.6.7', '6.7.0', 'E2', 'D4', '4.7', '4.7', '4.7', '4.6.7', '5.6', '4.7', '4.7', '4.7', 'F#2', '10.2', '5.6', 'A1', 'D4', 'A5', '5.9', '5.9', '4.6.7', 'A4', '5.9', '5.9', '5.9', '5.6.9.0', '9.0.2.5', '2.5.9', 'F2', 'A4', '5.9', '5.9', '4.6.7', '6.9', 'D5', '5.9', '5.9'

Created at ../output/LSTM_ABBA_1570771432/LSTM_output_final1
['F#5', 'E1', 'C#5', '4', '11.1.3.4.6', 'G#4', 'C6', '1.6', 'E1', '10.0.1', 'F#4', 'C#5', 'A4', '4', '11.1.6', '1.6', 'E1', 'G#4', '10.0.1', 'C#5', 'E4', '4', '11.1.3.4.6', '1.6', '10.11.1.4', 'F#4', 'C#5', '9.1', '11.1', '4.9', '9.1.4', '1.6', 'G#4', '10.1', '4.9', 'A0', 'C#5', '11.1.3.4.6', '1.6', 'A0', '10.1', '4.9', 'G#4', 'C#5', 'F#4', '11.1.6', '4.9', '1.6', 'A0', 'E4', '10.1', 'C#5', '11.1.3.4.6', '1.6', '9.10.1', 'C#5', 'A0', '11.1.6', '1.6', '10.1', 'A0', 'C#5', '11.1.3.4.6.8', '1.6', 'A0', '8.10.1', 'C#5', 'G#4', '11.1.6', '1.6', 'A0', 'F#4', '10.1', 'C#5', 'E4', '11.1.3.4.6', '1.6', '9.10.1', 'F#4', 'C#5', '11.3.6', 'B0', '11.1.6', '6.11', '1.6', '10.1', '6.11', 'G#4', 'B0', 'C#5', '11.1.3.4.6', '1.6', 'B0', '10.1', '6.11', 'B3', 'C#5', '11.1.6', '6.11', '1.6', 'B0', '10.1', 'C#5', '11.1.3.4.6', '6.11', '1.6', '10.11.1', '6.11', 'C#5', 'B0', '11.1.6', '6.11', '1.6', '10.1', 'B0', '11.1.6', '11.1.3.4.6', '6.11', '1.

Created at ../output/LSTM_ABBA_1570771432/LSTM_output_final3
['F#2', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', '2.6', 'F#2', 'C2', 'F#2', 'F#2', 'F#2', 'D2', 'F#2', '0.6', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', 'F#2', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', '2.6', 'F#2', 'C2', 'F#2', 'F#2', 'F#2', 'D2', 'F#2', '0.6', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', 'F#2', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', '2.6', 'F#2', 'C2', 'F#2', 'F#2', 'F#2', 'D2', 'F#2', '0.6', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', 'F#2', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', '2.6', 'F#2', 'C2', 'F#2', 'F#2', 'F#2', 'D2', 'F#2', '0.6', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', 'F#2', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', '2.6', 'F#2', 'C2', 'F#2', 'F#2', 'F#2', 'D2', 'F#2', '0.6', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', 'F#2', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', '2.6', 'F#2', 'C2', 'F#2', 'F#2', 'F#2', 'D2', 'F#2', '0.6', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', 'F#2', 'F#2', 'D2', 'F#2', 'F#2', 'C2', 'F#2', '2.

Created at ../output/LSTM_ABBA_1570771432/LSTM_output_final5
['C#4', '6.9', '6.11', 'F#1', 'G#4', 'F#2', 'C#4', 'F#4', 'C#4', 'A4', '2.6', '6.11', '4.8', 'G#4', 'F#1', 'D5', 'D4', 'F#4', '6.11', 'D2', '9.2', 'F#4', 'F#2', 'G#4', 'E4', '8.9', '6.9', 'A3', 'F#1', 'D4', 'C#4', 'E4', 'E4', 'D4', 'C#4', '6.9', 'F1', 'B4', 'C#4', 'A4', 'F#2', 'F#2', 'D2', 'C#4', '10.1', 'E4', '11.2.6', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', '9.1', 'C#5', 'E4', 'F#4', 'F#4', '6.11', 'E4', 'C#4', 'F#2', 'E4', 'E4', 'E4', '6.9', 'E4', '2.6', 'D5', 'E4', 'E4', 'F#5', 'E1', 'F#4', 'E4', 'E4', 'B3', 'E1', 'E4', 'E4', 'G#4', 'E4', 'A5', 'F#2', 'E4', 'C#2', '6.9', 'A3', 'D5', 'E4', 'F#4', 'C#4', 'E4', '10.1', 'E4', 'E1', 'B1', 'D4', 'E4', 'E4', 'E4', 'E4', 'E5', 'G#4', 'E1', '11.2', 'G#4', '9.1.4', '11.2.6', 'D5', 'F#2', '11.2', 'B3', '4.10', 'F#4', '10.1', 'D2', '9.2', '1.4', 'F#4', 'F#4', '6.10.1', 'D2', '6.10.11.1', 'F#4', 'F#4', '9.1.4', '9.2', '6.10.1', '9.1.4', '9.1.4', '9.2', '6.10.1', 'A4', 'F#4', '9.1', '1.4',

Created at ../output/LSTM_ABBA_1570771432/LSTM_output_final7
['E5', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', 'C#5', 'E4', 'A4', '2.4', '2.4', '1.4', '1.4', 'F#4', 'D5', '9.2', '9.2', '9.2', '9.2', 'C#5', '9.1', '9.1', '9.1', '9.1', 'E5', 'G4', 'B4', '11.4', '11.4', '11.4', '11.4', 'D5', '11.2', '11.2', '11.2', '11.2', 'B4', 'E5', 'E4', 'C#5', '6.9.0', 'B4', 'F#2', 'D5', '9.2', 'E5', '9.2', 'E5', '9.2', '9.2', '9.1', '9.2', '9.1', '9.2', '9.1', '9.2', '4.9', '4.9', '9.2', '11.2', '11.2.5', '2.6.9', '2.5', '11.2', '11.2', '11.2', '11.2', '11.2', '11.4', '11.2', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '1.4', '1.4', '1.4', '1.4', '1.4', '1.4', '1.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.2', '1.4', '1.4', '1.4', '1.4', '1.4', '1.4', '1.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '1.4', '1.4', '1.4', '1.4', '1.4', '1.4', '1.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', '11.4', 

Created at ../output/LSTM_ABBA_1570771432/LSTM_output_final9


Alternatively, I can run this script to convert all of the models into midi files and select my favourite from a much larger album.

In [23]:
# Have each model make a song
count = 0
filepaths = glob.glob(outputDest + "*.hdf5")
for model_path in filepaths:
    print("Composing from %s" % model_path)
    model.load_weights(model_path)
    prediction_output = generate_notes(model, network_input, possibleNotes, possibleOffsets, possibleDurations)
    create_midi(prediction_output, outputDest + 'LSTM_output_' + str(count))
    print(outputDest + 'LSTM_output_' + str(count))
    count += 1